# FAKE NEWS DETECTION SYSTEM

###### importation of important libraries

In [1]:
%config IPCompleter.greedy=True

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import spacy
from sklearn.linear_model import LogisticRegression

C:\Users\HP\anaconda3\envs\snowflakes\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### Reading the csv files into a dataframe and cleaning the data to be used for traaining

In [3]:
df1 = pd.read_csv('fake_or_real_news.csv')

In [4]:
df1.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
df1 = df1.drop('Unnamed: 0', axis=1)
df1.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
df1['label'] = df1['label'].astype('category')

In [7]:
df1.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
df1['label'] = df1['label'].map({'REAL':1, 'FAKE':0})
df1.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [9]:
df2 = pd.read_csv('data.csv')
df2.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [10]:
df2 = df2.drop('URLs',axis=1)
df2.head()

,Headline,Body,Label
0,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [11]:
df2.rename(columns = {'Headline':'title', 'Body':'text', 'Label':'label'}, inplace=True)
df2.head()

,title,text,label
0,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [12]:
df2['label'] = df2['label'].astype('category')

In [13]:
df2.dropna(how='any', axis=0, inplace=True)

In [14]:
df4 = pd.read_csv('Fake.csv')
df4.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [15]:
df4 = df4.drop(['subject', 'date'],axis=1)
df4.head()

,title,text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...


In [16]:
df4['label'] = 0
df4.head()

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0


In [17]:
df5 = pd.read_csv('True.csv')
df5.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [18]:
df5 = df5.drop(['subject', 'date'],axis=1)
df5.head()

,title,text
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...


In [19]:
df5['label'] = 1
df5.head()

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


##### Creating a manual testing csv file

In [20]:
df_fake_manual_testing = df4.tail(100)
for i in range(23480, 23380, -1):
    df4.drop([i], axis=0, inplace=True)

df_true_manual_testing = df5.tail(100)
for c in range(21416, 21316, -1):
    df5.drop([c], axis=0, inplace=True)
    
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)

df_manual_testing.to_csv('Manual testing.csv')

In [21]:
df1.shape, df2.shape, df4.shape, df5.shape

((6335, 3), (3988, 3), (23381, 3), (21317, 3))

##### Concatenating the dataframes that are to used for training and performing additional cleaning on it

In [22]:
main_df = pd.concat([df1, df2, df4, df5])
main_df.reset_index(inplace=True)

In [23]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55021 entries, 0 to 55020
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   55021 non-null  int64 
 1   title   55021 non-null  object
 2   text    55021 non-null  object
 3   label   55021 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.7+ MB


In [24]:
main_df['label'] = main_df['label'].astype('category')

In [25]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55021 entries, 0 to 55020
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   index   55021 non-null  int64   
 1   title   55021 non-null  object  
 2   text    55021 non-null  object  
 3   label   55021 non-null  category
dtypes: category(1), int64(1), object(2)
memory usage: 1.3+ MB


In [26]:
main_df['label'].value_counts()

0    28665
1    26356
Name: label, dtype: int64

In [27]:
main_df.drop('title', axis=1, inplace=True)

In [28]:
main_df.head()

,index,text,label
0,0,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,1,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,2,U.S. Secretary of State John F. Kerry said Mon...,1
3,3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,4,It's primary day in New York and front-runners...,1


In [29]:
main_df.drop(['index'], axis=1, inplace=True)

##### Next we must shuffle our dataset

In [30]:
main_df = main_df.sample(frac=1)
main_df.head(20)

,text,label
35854,(Reuters) - A U.S. appeals court on Wednesday ...,1
52351,MADRID (Reuters) - Spanish police have occupie...,1
19872,Mark Levin dropped a bombshell on his show ton...,0
1721,In several surreptitiously recorded video conv...,1
48818,ADEN/GENEVA (Reuters) - The Saudi-led military...,1
52759,CAIRO (Reuters) - Egyptian police arrested sev...,1
12603,Donald Trump is now demanding that any probe i...,0
34072,WASHINGTON (Reuters) - U.S. Senate Republican ...,1
23796,"Dana White, Ultimate Fighting Championship Pre...",0
14026,After evidence has been uncovered that could p...,0


##### Loading spacy medium english model

In [31]:
nlp = spacy.load('en_core_web_md')

C:\Users\HP\anaconda3\envs\snowflakes\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_md' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [32]:
main_df.head(20)

,text,label
35854,(Reuters) - A U.S. appeals court on Wednesday ...,1
52351,MADRID (Reuters) - Spanish police have occupie...,1
19872,Mark Levin dropped a bombshell on his show ton...,0
1721,In several surreptitiously recorded video conv...,1
48818,ADEN/GENEVA (Reuters) - The Saudi-led military...,1
52759,CAIRO (Reuters) - Egyptian police arrested sev...,1
12603,Donald Trump is now demanding that any probe i...,0
34072,WASHINGTON (Reuters) - U.S. Senate Republican ...,1
23796,"Dana White, Ultimate Fighting Championship Pre...",0
14026,After evidence has been uncovered that could p...,0


##### Removing empty strings from the dataset

In [33]:
main_df[main_df['text'] == " "]

,text,label
26578,,0
29697,,0
22164,,0
23263,,0
22229,,0
...,...,...
6328,,0
22928,,0
21626,,0
23122,,0


In [34]:
main_df = main_df[main_df['text'] != " "]

In [35]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54358 entries, 35854 to 30587
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   text    54358 non-null  object  
 1   label   54358 non-null  category
dtypes: category(1), object(1)
memory usage: 902.5+ KB


##### Removing punctuations, numbers nd urls from the texts and coverting the text to lower

In [36]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [37]:
main_df['text'] = main_df['text'].apply(wordopt)

##### Creating a custum vectorized function to remove stopwords and return all words to their baseform

In [38]:
def clean(text):
    text = " ".join([token.lemma_ for token in nlp(text) if not token.is_stop])
    return text

clean_vector = np.vectorize(clean, doc='Vectorized "clean"', otypes=[list])

In [39]:
main_df['text'].head()

35854     reuters    a u s  appeals court on wednesday ...
52351    madrid  reuters    spanish police have occupie...
19872    mark levin dropped a bombshell on his show ton...
1721     in several surreptitiously recorded video conv...
48818    aden geneva  reuters    the saudi led military...
Name: text, dtype: object

##### cleaning the dataset

In [40]:
main1 = main_df['text'].head(27000).apply(clean_vector)

In [41]:
main2 = main_df['text'].tail(27000).apply(clean_vector)

In [42]:
main_df2 = pd.concat([main1, main2])

In [43]:
main_df2.info()

<class 'pandas.core.series.Series'>
Int64Index: 54000 entries, 35854 to 30587
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
54000 non-null  object
dtypes: object(1)
memory usage: 843.8+ KB


In [44]:
main_df2.head()

35854      reuters     u s   appeal court wednesday rev...
52351    madrid   reuters     spanish police occupy cat...
19872    mark levin drop bombshell tonight muslim terro...
1721     surreptitiously record video conversation   sc...
48818    aden geneva   reuters     saudi lead military ...
Name: text, dtype: object

In [45]:
main_df.rename(columns = {'text':'Text'}, inplace=True)

In [46]:
main_df = main_df.join(main_df2, how='right')

In [47]:
main_df.drop(['Text'], axis=1, inplace=True)

In [48]:
main_df.head()

,label,text
35854,1,reuters u s appeal court wednesday rev...
52351,1,madrid reuters spanish police occupy cat...
19872,0,mark levin drop bombshell tonight muslim terro...
1721,1,surreptitiously record video conversation sc...
48818,1,aden geneva reuters saudi lead military ...


##### Splitting the data into training and testing data, then vectorizing the dataset for easy processing 

In [49]:
X_train, X_test, y_train, y_test = train_test_split(main_df['text'], main_df['label'], test_size=0.3, 
                                                    stratify=main_df['label'], random_state=25)

In [50]:
tfVectorizer = TfidfVectorizer()

In [51]:
X_train = tfVectorizer.fit_transform(X_train)
X_test = tfVectorizer.transform(X_test)

##### Training the model using logistic regression

In [52]:
LR = LogisticRegression(random_state=25)

In [53]:
LR.fit(X_train, y_train)

LogisticRegression(random_state=25)

In [54]:
y_pred = LR.predict(X_test)

In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      8345
           1       0.96      0.94      0.95      7855

    accuracy                           0.95     16200
   macro avg       0.95      0.95      0.95     16200
weighted avg       0.95      0.95      0.95     16200



##### Creating a function to process inputed news text

In [56]:
def result(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_def_test["text"] = new_def_test["text"].apply(clean_vector)
    new_x_test = new_def_test["text"]
    new_xv_test = tfVectorizer.transform(new_x_test)
    y_pred = LR.predict(new_xv_test)
    
    return print("\n\n Prediction: {} ".format(result(y_pred)))

##### News input

In [ ]:
news = str(input())
manual_testing(news)